# Classification using Keras

The target values must be in a column of the dataframe, and must be a numerical datatype, so we must convert them before training.

IMPORTANT: Notice that we are importing `ImageDataGenerator` from `keras_preprocessing` instead of `keras.preprocessing`, because the people at Keras are a bunch of old reactionary dinosaurs.

See the discussion [here](https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1), and [here](https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1).

In [27]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

## Load data and preparation

The csv files `db.csv` contains all the metadata we have extracted from wikiart: every entry corresponds to an artwork, and the `_id` column contains the filename of the image in the `data/images/` directory associated to that artwork.

In [143]:
nrows = 5000
df = pd.read_csv("../data/db.csv",nrows=nrows,na_values="?")

Train/test split the database:

In [144]:
df_train, df_test = train_test_split(df, 
                                     test_size=0.1,
                                     shuffle=True)

In [145]:
df_train.head()

,_id,artistname,genre,image,image_size_data,style,title,year
1222,57727194edc2cb3880c22e6f,camille pissarro,landscape,https://uploads7.wikiart.org/images/camille-pi...,"[{'sizekb': 9, 'width': 210, 'height': 176, 'u...",impressionism,fields,1877
4691,57727fc0edc2cb3880eed7e6,mykola pymonenko,genre painting,https://uploads5.wikiart.org/images/mykola-pym...,"[{'sizekb': 14, 'width': 210, 'height': 264, '...",realism,laundry,NaN
4584,57727f8bedc2cb3880ee2bee,mykola pymonenko,genre painting,https://uploads3.wikiart.org/images/mykola-pym...,"[{'sizekb': 11, 'width': 210, 'height': 317, '...",realism,conversation,1912
797,577272e7edc2cb3880c6bb4c,niko pirosmani,genre painting,https://uploads6.wikiart.org/images/niko-piros...,"[{'sizekb': 8, 'width': 210, 'height': 211, 'u...",naïve art (primitivism),tatar - camel driver,NaN
4709,57727720edc2cb3880d3c21f,qi baishi,animal painting,https://uploads6.wikiart.org/images/qi-baishi/...,"[{'sizekb': 9, 'width': 210, 'height': 175, 'u...",ink and wash painting,shrimp,NaN


Let's initialize a dataimage generator: it is a nice interface towards many (pre)processing method in Keras, including some utilities for data augmentation.

We will use the amazing `flow_from_dataframe` function to serve the data we need.

If the files do not have an extension, run this in a shell:

    $ for f in *; do mv "$f" "$f.jpg"; done

### Classes

Decide here what feature we want to predict, and save in the `classes` set all the possible values: they are the values that appear at least once in the database.

In [183]:
feature = "style"

In [187]:
classes = set(df[feature])

The number of classes will be needed later, the NN must know what's the output dimension.

In [188]:
nclass = len(classes)

In [189]:
classes

{'abstract art',
 'abstract art,abstract expressionism',
 'abstract expressionism',
 'abstract expressionism,action painting',
 'academicism',
 'action painting',
 'art brut',
 'art informel',
 'art informel,automatic painting',
 'art informel,expressionism',
 'art informel,minimalism',
 'art informel,surrealism',
 'art nouveau (modern)',
 'art nouveau (modern),impressionism',
 'art singulier',
 'baroque',
 'classicism',
 'color field painting',
 'conceptual art',
 'conceptual art,environmental (land) art',
 'conceptual art,minimalism',
 'conceptual art,pop art',
 'concretism',
 'constructivism',
 'contemporary realism',
 'cubism',
 'cubism,expressionism',
 'cubism,futurism',
 'cubism,surrealism',
 'dada',
 'early renaissance',
 'early renaissance,international gothic',
 'environmental (land) art',
 'expressionism',
 'expressionism,muralism',
 'expressionism,pop art',
 'expressionism,social realism',
 'futurism',
 'gongbi',
 'hard edge painting',
 'hard edge painting,neoplasticism',
 '

### Image Size

The images will be scaled down to this size

In [190]:
img_size = (32,32)

## Create data generators

In [193]:
datagen = ImageDataGenerator(rescale=1/255,validation_split=0.2)

In [194]:
train_generator = datagen.flow_from_dataframe(
                                df_train,
                                directory="../data/images/",
                                x_col="_id",
                                has_ext=False,
                                target_size=img_size,
                                y_col=feature,
                                batch_size=32,
                                classes = classes)

Found 4500 images belonging to 80 classes.


In [195]:
test_generator = datagen.flow_from_dataframe(
                                df_test,
                                directory="../data/images/",
                                x_col="_id",
                                has_ext=False,
                                target_size=img_size,
                                y_col=feature,
                                batch_size=20,
                                shuffle=False,
                                classes=classes) 

Found 500 images belonging to 80 classes.


# Model Architecture

We'll start with the most basic architecture imaginable. 

It would be nice to use transfer learning.

In [196]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(*img_size,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nclass, activation='softmax'))

Pick an optimizer and compile the model

In [197]:
model.compile(
    optimizers.rmsprop(lr=0.0001, 
                       decay=1e-6),
    loss="categorical_crossentropy",
    metrics=["accuracy"])

# Training

In [198]:
model.fit_generator(generator=train_generator,
                    epochs=2)

Epoch 1/2
141/141 [==============================] - 49s 350ms/step - loss: 3.3666 - acc: 0.1953
Epoch 2/2
141/141 [==============================] - 46s 327ms/step - loss: 3.0616 - acc: 0.2853


# Evaluation

This returns the loss and accuracy

In [201]:
model.evaluate_generator(generator=test_generator)

[2.975468864440918, 0.3299999988079071]